# Using AI and NLP to Improve Your Cooking

Michael Humkey, Conor Marsten

In [1]:
import urllib
import urllib.request

In [2]:
url = "http://food2fork.com/api/search?"
args = {'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'q' : 'taco', 'sort' : 'r'}
data = urllib.parse.urlencode(args)
req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(req).read()

In [3]:
import json

In [4]:
response = json.loads(response)
recipes = response['recipes']

In [5]:
from bs4 import BeautifulSoup
import requests

In [7]:
def getReviews(recipes, i=0):

    recipe = recipes[i]
    url = "http://food2fork.com/api/get?"
    data = urllib.parse.urlencode({'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'rId' : recipe['recipe_id']})
    req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
    recipe = json.loads(urllib.request.urlopen(req).read())['recipe']
    bs =  BeautifulSoup(requests.get(recipe['source_url']).content, 'html.parser')
    reviews = []

    if (recipe['publisher'] == '101 Cookbooks'): #1
        reviews = bs.find_all(attrs={'class': 'card-body'})

    elif (recipe['publisher'] == 'BBC Good Food'):
        reviews = bs.find_all(attrs={'class': 'field-item even'})

    elif (recipe['publisher'] == 'Closet Cooking' 
           or recipe['publisher'] == 'Eats Well With Others' 
           or recipe['publisher'] == "Lisa's Kitchen"
           or recipe['publisher'] == 'A Spicy Perspective'
           or recipe['publisher'] == 'Naturally Ella'
           or recipe['publisher'] == 'Pastry Affair'):
        reviews = bs.find_all(attrs={'class': 'comment-body'})

    elif (recipe['publisher'] == 'Food Republic'):
        reviews = bs.find_all(attrs={'class': '_5mdd'}) #uses facebook comments plugin

    elif (recipe['publisher'] == 'PBS Food'):
        reviews = bs.find_all(attrs={'class': 'post-message'})

    elif (recipe['publisher'] == 'Simply Recipes'
          or recipe['publisher'] == 'Homesick Texan'
          or recipe['publisher'] == 'Tasty Kitchen'):
        reviews = bs.find_all(attrs={'class': 'comment-text'})

    elif (recipe['publisher'] == 'Two Peas and Their Pod'
          or recipe['publisher'] == 'Cookie and Kate'
          or recipe['publisher'] == "Elana's Pantry"
          or recipe['publisher'] == 'My Baking Addiction'
          or recipe['publisher'] == 'Smitten Kitchen'
          or recipe['publisher'] == 'Vintage Mixer'
          or recipe['publisher'] == 'Cookin Canuck'
          or recipe['publisher'] == 'Healthy Delicious'
          or recipe['publisher'] == 'Steamy Kitchen'
          or recipe['publisher'] == "What's Gaby Cooking"
          or recipe['publisher'] == 'Bunky Cooks'
          or recipe['publisher'] == 'Serious Eats'):
        reviews = bs.find_all(attrs={'class': 'comment-content'})

    elif (recipe['publisher'] == 'All Recipes'):
        reviews = bs.find_all(attrs={'class' : 'review-detail__link'})
        revList = []
        for review in reviews:
            revList.append(str(review).split(' ')[2].replace('href=\"', '').replace('\"', ''))
        reviews = []
        for url in revList:
            bs =  BeautifulSoup(requests.get(url).content, 'html.parser')
            reviews.append(bs.find(itemprop="reviewBody"))

    elif (recipe['publisher'] == 'Big Girls Small Kitchen'
          or recipe['publisher'] =='Jamie Oliver'
          or recipe['publisher'] == 'The Pioneer Woman'):
        reviews = bs.find_all(attrs={'data-role': 'message'})

    elif (recipe['publisher'] == 'Framed Cooks'
          or recipe['publisher'] == 'Picky Palate'):
        reviews = bs.find_all(attrs={'class': 'comment even thread-even depth-1'})

    elif (recipe['publisher'] == 'Bon Appetit'):
        reviews = bs.find_all(attrs={'class': 'review-body'})

    elif (recipe['publisher'] == 'Epicurious'):
        reviews = bs.find_all(attrs={'class': 'review-text'})

    elif (recipe['publisher'] == 'Cookstr'):
        reviews = bs.find_all(attrs={'class': 'commentText'})

    elif (recipe['publisher'] == 'Panini Happy'):
        #careful, sloppy html
        reviews = bs.find_all(attrs={'class': 'format_text'})

    elif (recipe['publisher'] == 'Real Simple'):
        reviews = bs.find_all(attrs={'class': 'comment_txt'})
        
    elif (recipe['publisher'] == 'Chow'):
        reviews = bs.find_all(itemprop="comment")

    elif (recipe['publisher'] == 'Delishhh'):
        reviews = bs.find_all(attrs={'class': 'commentmeta'})

    elif (recipe['publisher'] == 'Food Network'):
        reviews = bs.find_all(attrs={'class': 'gig-comment-body'})    
    else:
        print(recipe['publisher'])
        getReviews(recipes, i+1)
        
    return reviews, recipe

In [10]:
reviews, recipe = getReviews(recipes)    
# if type(reviews[0]) is not str:
revs = [rv.get_text().replace('\n', ' ').replace('\r', '') for rv in reviews]
# else:
#     revs = [rv.replace('\n', ' ').replace('\r', '') for rv in reviews]

In [11]:
print(type(reviews[0]))
print(reviews[0])

<class 'bs4.element.Tag'>
<p itemprop="reviewBody">
                This is my recipe and I am thrilled you all are loving it!  Here is the recipe for Bill Echols Taco Seasoning.  It really makes this recipe.
1 tablespoon chili powder 
1/4 teaspoon garlic powder 
1/4 teaspoon onion powder 
1/4 teaspoon crushed red pepper flakes 
1/4 teaspoon dried oregano 
1/2 teaspoon paprika 
1 1/2 teaspoons ground cumin 
1 teaspoon sea salt 
1 teaspoon black pepper
            </p>


## Checking for foods

In [16]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')

In [18]:
recipe['publisher']

'All Recipes'

In [19]:
recipe

{'f2f_url': 'http://food2fork.com/view/28924',
 'image_url': 'http://static.food2fork.com/9843414ab7.jpg',
 'ingredients': ['1 onion, chopped',
  '1 (16 ounce) can chili beans',
  '1 (15 ounce) can black beans',
  '1 (15 ounce) can whole kernel corn, drained',
  '1 (8 ounce) can tomato sauce',
  '1 (12 fluid ounce) can or bottle beer',
  '2 (10 ounce) cans diced tomatoes with green chilies, undrained',
  '1 (1.25 ounce) package taco seasoning',
  '3 whole skinless, boneless chicken breasts',
  'shredded Cheddar cheese (optional)',
  'sour cream (optional)',
  'crushed tortilla chips (optional)'],
 'publisher': 'All Recipes',
 'publisher_url': 'http://allrecipes.com',
 'recipe_id': '28924',
 'social_rank': 99.99999999999945,
 'source_url': 'http://allrecipes.com/Recipe/Slow-Cooker-Chicken-Taco-Soup/Detail.aspx',
 'title': 'Slow Cooker Chicken Taco Soup'}

In [23]:
import nltk
from nltk import word_tokenize, pos_tag
text = word_tokenize(reviews[0].get_text())
tagged_text = nltk.pos_tag(text)
print(tagged_text)

[('This', 'DT'), ('is', 'VBZ'), ('my', 'PRP$'), ('recipe', 'NN'), ('and', 'CC'), ('I', 'PRP'), ('am', 'VBP'), ('thrilled', 'VBN'), ('you', 'PRP'), ('all', 'DT'), ('are', 'VBP'), ('loving', 'VBG'), ('it', 'PRP'), ('!', '.'), ('Here', 'RB'), ('is', 'VBZ'), ('the', 'DT'), ('recipe', 'NN'), ('for', 'IN'), ('Bill', 'NNP'), ('Echols', 'NNP'), ('Taco', 'NNP'), ('Seasoning', 'NNP'), ('.', '.'), ('It', 'PRP'), ('really', 'RB'), ('makes', 'VBZ'), ('this', 'DT'), ('recipe', 'NN'), ('.', '.'), ('1', 'CD'), ('tablespoon', 'NN'), ('chili', 'NN'), ('powder', 'NN'), ('1/4', 'CD'), ('teaspoon', 'NN'), ('garlic', 'JJ'), ('powder', 'NN'), ('1/4', 'CD'), ('teaspoon', 'NN'), ('onion', 'NN'), ('powder', 'NN'), ('1/4', 'CD'), ('teaspoon', 'NN'), ('crushed', 'VBD'), ('red', 'JJ'), ('pepper', 'NN'), ('flakes', 'VBZ'), ('1/4', 'CD'), ('teaspoon', 'NN'), ('dried', 'VBD'), ('oregano', 'JJ'), ('1/2', 'CD'), ('teaspoon', 'NN'), ('paprika', 'VBD'), ('1', 'CD'), ('1/2', 'CD'), ('teaspoons', 'NNS'), ('ground', 'NN'), 

In [49]:
import sqlite3
conn = sqlite3.connect('usda.sql3')
excluded_words = ['recipe', 'powder', 'ground', 'sea']
out =[]

for rv in revs:
    text = word_tokenize(reviews[0].get_text())
    tagged_text = nltk.pos_tag(text)
    for tt in tagged_text:
        if tt[1] == 'NN' and tt[0] not in excluded_words:
            search = '%' + tt[0] + '%'
            c = conn.cursor()
            c.execute('SELECT id, short_desc FROM food WHERE short_desc LIKE ?', (search,))
            values = c.fetchall()
            if len(values) != 0:
                for sentence in rv.split('.'):
                    if tt[0] in sentence and sentence not in out:
                        out.append(sentence)
#             for row in values:
#                 print(tt, row[0], row[1])
for x in out:
    print(x)

 1 tablespoon chili powder  1/4 teaspoon garlic powder  1/4 teaspoon onion powder  1/4 teaspoon crushed red pepper flakes  1/4 teaspoon dried oregano  1/2 teaspoon paprika  1 1/2 teaspoons ground cumin  1 teaspoon sea salt  1 teaspoon black pepper             
 In addition to these toppings  I added a very small amount of thinly sliced green onion and fresh cilantro
 However, I took a chance and used Heineken for this recipe as well as the taco seasoning and tomatoes with chilies and the soup was DELICIOUS
  Used pinto beans instead of chili beans and added a full 28 oz can of diced tomatoes
 I added some chili powder, cumin, flaked red pepper, salt, and garlic powder
  I left out the onion because my family doesn't like it


## Performing sentiment analysis

In [50]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

In [51]:
analyzer = sia()
scored = []
for sent in out:
    res = analyzer.polarity_scores(sent)
    tup = (sent, res)
    scored.append(tup)

In [52]:
def getKey(tup):
    return tup[1]['compound']

scored = list(reversed(sorted(scored, key=getKey)))

In [53]:
scored

[(' However, I took a chance and used Heineken for this recipe as well as the taco seasoning and tomatoes with chilies and the soup was DELICIOUS',
  {'compound': 0.8192, 'neg': 0.0, 'neu': 0.711, 'pos': 0.289}),
 (' In addition to these toppings  I added a very small amount of thinly sliced green onion and fresh cilantro',
  {'compound': 0.3182, 'neg': 0.0, 'neu': 0.874, 'pos': 0.126}),
 (' 1 tablespoon chili powder  1/4 teaspoon garlic powder  1/4 teaspoon onion powder  1/4 teaspoon crushed red pepper flakes  1/4 teaspoon dried oregano  1/2 teaspoon paprika  1 1/2 teaspoons ground cumin  1 teaspoon sea salt  1 teaspoon black pepper             ',
  {'compound': 0.0258, 'neg': 0.074, 'neu': 0.663, 'pos': 0.263}),
 (' I added some chili powder, cumin, flaked red pepper, salt, and garlic powder',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 ('  Used pinto beans instead of chili beans and added a full 28 oz can of diced tomatoes',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0,